In [1]:
# find the earliest campaign with a video, international shipping,
# reward dates, etc -- goal is to find a date range in which to restrict 
# my prediction.
# can I then use faq, rewards, etc? 
#
# remove campaigns with 0 launch time (~5)
# add live campaigns -- check my text.
#
# then excluded cancelled and suspended campaigns

In [2]:
import requests
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode

In [3]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [5]:
# search all of my scraped pages for videos, find the earliest with a video by this critera.
# check for false negatives -- success/failure for these videos

In [6]:
has_video_df = pd.read_pickle('has_video_df.save')

In [10]:
has_video_df.head()

,has_video,URL,launch_t
1,True,https://www.kickstarter.com/projects/213558405...,1417370930
2,True,https://www.kickstarter.com/projects/647561166...,1416402978
3,True,https://www.kickstarter.com/projects/158735721...,1417132986
4,True,https://www.kickstarter.com/projects/318653143...,1415682456
5,True,https://www.kickstarter.com/projects/vermontco...,1414015049


In [23]:
import time
time.ctime(video_campaign_ts[127314])

'Wed Apr 22 18:07:53 2009'

In [37]:
has_video_df.sort(columns='launch_t',ascending=False).head(10)

,has_video,URL,launch_t
176574,True,https://www.kickstarter.com/projects/artofdino...,1417543710
146301,False,https://www.kickstarter.com/projects/117811785...,1417542539
168775,False,https://www.kickstarter.com/projects/168828926...,1417542301
168681,False,https://www.kickstarter.com/projects/975131435...,1417542069
155536,True,https://www.kickstarter.com/projects/189058307...,1417541211
176463,False,https://www.kickstarter.com/projects/161943526...,1417541117
166318,False,https://www.kickstarter.com/projects/197313541...,1417541022
155047,True,https://www.kickstarter.com/projects/142563967...,1417540997
155108,True,https://www.kickstarter.com/projects/323324338...,1417540400
171372,False,https://www.kickstarter.com/projects/tesladyne...,1417539863


In [91]:
print has_video_df.ix[171372,'URL']

https://www.kickstarter.com/projects/tesladyne/the-bureau-of-infernal-affairs?ref=category


In [75]:
with con:
    cur = con.cursor()
    cur.execute("SELECT Project_html FROM Scraped_pages WHERE Id = 176463")
    rows = cur.fetchall()
    cur.close()

In [76]:
project_html = rows[0][0]
print project_html

<div class="NS_projects__description_section">
<div class="py6 bg-white">
<div class="container-flex px2">
<div class="row">
<div class="col col-8">
<div class="mb6">
<h3 class="normal mb4 mobile-hide">
About this project
</h3>
</div>
<div class="full-description js-full-description responsive-media formatted-lists">
<a href="http://Kids%20Art%20Museum,%20Monterey,%20CA" target="_blank"><div class="template asset" contenteditable="false" data-edit_url="/projects/1619435265/give-me-a-museum-and-i-will-fill-it-kidsartmuseum/assets/2954746/edit" data-id="2954746">
<figure>
<img alt="From here -  to There" class="fit" src="https://ksr-ugc.imgix.net/assets/002/954/746/3c238f918072ac1bfac5e34ea2b90618_original.jpg?v=1416693935&amp;w=680&amp;h=&amp;fit=max&amp;auto=format&amp;q=92&amp;s=d5492c2e333c3261cf2c238d0f236bde">
<figcaption>From here -  to There</figcaption>
</figure>

</div>


</a><h1><b>"Give me a Museum, and I will fill it."</b></h1><p>Pablo Picasso &amp; Young Artists of Monterey

In [84]:
project_request = requests.get('https://www.kickstarter.com/projects/1178117855/the-withnail-coat?ref=category')
#project_request = requests.get('https://www.kickstarter.com/projects/1619435265/give-me-a-museum-and-i-will-fill-it-kidsartmuseum/description')
project_raw = project_request.text
print project_raw

<!DOCTYPE html>
<html class="projects_show  fontface no-js" lang="en">
<script>
  //<![CDATA[
    docElement = this.document.documentElement;
    docElement.className=docElement.className.replace(/\bno-js\b/,'') + 'js';
  //]]>
</script><title>The Withnail Coat by Matthew Trevor-Roper &mdash; Kickstarter</title><meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">
<meta property="twitter:app:id:iphone" content="596961532"/>
<meta property="twitter:app:name:iphone" content="Kickstarter"/>
<meta property="twitter:app:url:iphone" content="ksr://www.kickstarter.com/projects/1178117855/the-withnail-coat"/>
<meta name="apple-itunes-app" content="app-id=596961532, app-argument=ksr://www.kickstarter.com/projects/1178117855/1870211446?app_banner=1&amp;ref=category"/>
<meta name="video_height" content="480"/>
<meta name="video_width" content="640"/>
<meta name="video_type" content="application/x-shockwave-flash"/>
<meta name="title" content="

In [86]:
match = re.search("data-has-video=\"true\"",project_raw)
if match:
    print match.group()

data-has-video="true"
